In [5]:
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from datetime import datetime, timezone
import pytz
import calendar


stock = ["EBAY", "AAPL", "AMZN", "CSCO", "MSFT"]
data = []
for s in stock:
    url = "https://api.iextrading.com/1.0/stock/" + s + "/chart/1d"
    response = requests.get(url).json()
    for x in response:
        x['stock'] = s
    data.extend(response)

df = pd.DataFrame(data)



df2 = df[['stock', 'date', 'minute', 'average']].copy()


df2['DateTime'] = df2['date'] + ' ' + df2['minute']
df2['DTObject'] = pd.to_datetime(df2['DateTime'])


DT_converted = []
local = pytz.timezone("EST")
for idx, row in df2.iterrows():
    DT_temp = local.localize(row['DTObject'], is_dst=None)
    DT_converted.append(DT_temp.astimezone(pytz.utc))

df2['DT_UTC'] = DT_converted




UTC_Unix = []
for idx, row in df2.iterrows():
    UTC_Unix.append(calendar.timegm(row['DT_UTC'].timetuple()))
    
df2['UTC_Unix'] = UTC_Unix



df3 = df2[['stock', 'UTC_Unix', 'average']].copy()




from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()



rds_connection_string = "root:ggZtOs8585@127.0.0.1/raw_data_db"
engine = create_engine(f'mysql://{rds_connection_string}')

with engine.connect() as con:
    con.execute(f'drop table if exists Stock;')
    con.execute(f'''create table if not exists Stock
               (Stock varchar(4),
               UTC_Unix double,
               average decimal(13,5),
               PRIMARY KEY (UTC_Unix, Stock),
               FOREIGN KEY (Stock) REFERENCES ticker_data(ticker)
               );''')

df3.to_sql(name='Stock', con=engine, if_exists='append', index=False)
